In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import Counter
import shutil
from google.colab import files
from huggingface_hub import notebook_login, HfApi, Repository
import re
import nltk
from nltk.corpus import stopwords

# NLTK stopwords listesini indir
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df_1 = pd.read_csv('/content/gdrive/MyDrive/Teknofest 2024 Türkçe Doğal Dil İşleme - Data Destroyers Takımı/text_nlp_datasets/sentiment_data.csv')


In [ ]:
df_1.head()

,Unnamed: 0,label,temizlenmiş_yorum,sentiment
0,0,1,yazarın okuduğum kitabı sultanı öldürmekti gün...,Negatif
1,1,1,başladığınız zaman kopamıyorsunuz günde bitird...,Negatif
2,2,1,kadınlara tavsidedir okuyun erkeklere tavsiyed...,Negatif
3,3,1,kesinlikle tavsiye ediyorum günlük hayattan uz...,Negatif
4,4,1,kesinlikle mutlaka okuması gereken kitap elif ...,Negatif


In [ ]:
df_1.shape

(449781, 4)

In [ ]:
df_1.head(
)

,label,temizlenmiş_yorum,sentiment
0,1,yazarın okuduğum kitabı sultanı öldürmekti gün...,Negatif
1,1,başladığınız zaman kopamıyorsunuz günde bitird...,Negatif
2,1,kadınlara tavsidedir okuyun erkeklere tavsiyed...,Negatif
3,1,kesinlikle tavsiye ediyorum günlük hayattan uz...,Negatif
4,1,kesinlikle mutlaka okuması gereken kitap elif ...,Negatif


In [ ]:
df_1 = df_1.iloc[:, 1:]

In [ ]:
df_1.isnull().sum()

,0
label,0
temizlenmiş_yorum,92
sentiment,0


In [ ]:
df_1.dropna(inplace=True)

In [ ]:
  word_counts = Counter(' '.join(df['temizlenmiş_yorum']).lower().split())

  def preprocess_text(text, word_counts):
      if not isinstance(text, str):
          return text

      # 1. Küçük harfe çevirme
      text = text.lower()

      # 2. Sayıları kaldırma
      text = re.sub(r'\d+', '', text)

      # 3. Fazla boşlukları kaldırma
      text = re.sub(r'\s+', ' ', text).strip()

      # 4. Noktalama işaretlerini kaldırma
      text = re.sub(r'[^\w\s]', '', text)

      # 5. Durdurma kelimelerini kaldırma
      stop_words = set(stopwords.words('turkish'))
      text = ' '.join([word for word in text.split() if word not in stop_words])

      # 6. Özel karakterleri kaldırma
      text = re.sub(r'[^a-zA-ZçğıöşüÇĞİÖŞÜ\s]', '', text)


      return text

  # Veriyi ön işleme tabi tut ve temizlenmiş_yorum sütununu oluştur
  df['temizlenmiş_yorum'] = df['temizlenmiş_yorum'].apply(lambda x: preprocess_text(x, word_counts))

0         yazarın okuduğum kitabı sultanı öldürmekti gün...
1         başladığınız zaman kopamıyorsunuz günde bitird...
2         kadınlara tavsidedir okuyun erkeklere tavsiyed...
3         kesinlikle tavsiye ediyorum günlük hayattan uz...
4         kesinlikle mutlaka okuması gereken kitap elif ...
                                ...                        
449848    i phone pro max yurt dışı cihazım temmuz resmi...
449849    i phone pro yurt dışı cihazım temmuz resmi kay...
449850    turkcell firması ek faturalı tarifeden aydınlı...
449851    turkcell yurt dışı çevresinde bulunduğumda yur...
449852    i phone pro max cihazımdaki hattımı dün kayıt ...
Name: temizlenmiş_yorum, Length: 449781, dtype: object


In [ ]:
word_counts = Counter(' '.join(df['temizlenmiş_yorum']).lower().split())

def preprocess_text(text, word_counts):
    if not isinstance(text, str):
        return text

    # 1. Küçük harfe çevirme
    text = text.lower()

    # 2. Sayıları kaldırma
    text = re.sub(r'\d+', '', text)

    # 3. Noktalama işaretlerini kaldırma ve onları bir boşlukla değiştirme
    text = re.sub(r'[^\w\s]', '', text)

    # 4. Durdurma kelimelerini kaldırma
    stop_words = set(stopwords.words('turkish'))
    text = ' '.join([word for word in text.split() if word not in stop_words])

    # 5. Özel karakterleri kaldırma
    text = re.sub(r'[^a-zA-ZçğıöşüÇĞİÖŞÜ\s]', '', text)

    # 6. Fazla boşlukları kaldırma (birden fazla boşluk varsa tek boşlukla değiştirme)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Veriyi ön işleme tabi tut ve temizlenmiş_yorum sütununu oluştur
df['temizlenmiş_yorum'] = df['temizlenmiş_yorum'].apply(lambda x: preprocess_text(x, word_counts))

In [ ]:
df.to_csv('text_data.csv')

In [ ]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# 1. Veriyi Hazırlama
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Sentiment etiketlerini sayısal değerlere dönüştürme
label_encoder = LabelEncoder()
df_1['label'] = label_encoder.fit_transform(df_1['sentiment'])

# 2. Model ve Tokenizer Seçimi
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
max_len = 128

# 3. Hyperparametre Ayarı
batch_size = 16  # Büyük veri seti için daha büyük batch size kullanılabilir
learning_rate = 2e-5
epochs = 3  # Daha fazla epoch kullanabilirsiniz, ancak eğitim süresi artacaktır

# 4. Eğitim ve Doğrulama Ayrımı
train_texts, val_texts, train_labels, val_labels = train_test_split(df_1['temizlenmiş_yorum'].to_numpy(), df_1['label'].to_numpy(), test_size=0.2)

train_dataset = TextDataset(
    texts=train_texts,
    labels=train_labels,
    tokenizer=tokenizer,
    max_len=max_len
)

val_dataset = TextDataset(
    texts=val_texts,
    labels=val_labels,
    tokenizer=tokenizer,
    max_len=max_len
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 2. Modeli ve Optimizer'ı Hazırlama
model = AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-cased", num_labels=len(label_encoder.classes_))
optimizer = AdamW(model.parameters(), lr=learning_rate, correct_bias=False)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# 6. GPU Kullanımı
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 3. Modeli Eğitme
def train_epoch(model, data_loader, optimizer, device, scheduler):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["label"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        preds = outputs.logits.argmax(dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)

def eval_model(model, data_loader, device):
    model = model.eval()
    losses = []
    correct_predictions = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["label"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            preds = outputs.logits.argmax(dim=1)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses), acc, precision, recall, f1

# Eğitim ve Doğrulama Döngüsü
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    train_acc, train_loss = train_epoch(model, train_loader, optimizer, device, scheduler)
    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss, val_acc_metric, val_precision, val_recall, val_f1 = eval_model(model, val_loader, device)
    print(f'Validation loss {val_loss} accuracy {val_acc}')
    print(f'Validation Accuracy: {val_acc_metric}')
    print(f'Validation Precision: {val_precision}')
    print(f'Validation Recall: {val_recall}')
    print(f'Validation F1 Score: {val_f1}')

print("Eğitim tamamlandı.")


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3
Train loss 0.14655894381078016 accuracy 0.9475970879858568
              precision    recall  f1-score   support

     Negatif       0.92      0.71      0.80      1403
        Nötr       0.96      0.96      0.96     40981
     Pozitif       0.95      0.96      0.96     47554

    accuracy                           0.96     89938
   macro avg       0.95      0.88      0.91     89938
weighted avg       0.96      0.96      0.96     89938

Validation loss 0.12535997487013784 accuracy 0.9562921123440592
Validation Accuracy: 0.9562921123440592
Validation Precision: 0.9561830822946087
Validation Recall: 0.9562921123440592
Validation F1 Score: 0.9560023936360369
Epoch 2/3
Train loss 0.09297495052335811 accuracy 0.9680751408613179
              precision    recall  f1-score   support

     Negatif       0.82      0.77      0.79      1403
        Nötr       0.97      0.95      0.96     40981
     Pozitif       0.95      0.97      0.96     47554

    accuracy                           0

In [ ]:
import os

# Model ve Tokenizer'ı kaydetmek için dizin oluşturma
model_save_path = "./saved_model"
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

# Modelin ağırlıklarını kaydetme
model.save_pretrained(model_save_path)

# Tokenizer'ı kaydetme
tokenizer.save_pretrained(model_save_path)

print(f"Model ve tokenizer {model_save_path} dizinine kaydedildi.")


Model ve tokenizer ./saved_model dizinine kaydedildi.


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGr

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch

# Modeli ve tokenizer'ı tanımlayın (bu örnekte eğitilmiş bir model varsayılmıştır)
model_save_path = "./saved_model"
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

# Modelin ağırlıklarını kaydetme
torch.save(model.state_dict(), os.path.join(model_save_path, "model.pkl"))

# Tokenizer'ı kaydetme
tokenizer.save_pretrained(model_save_path)

print(f"Model ve tokenizer {model_save_path} dizinine kaydedildi.")


Model ve tokenizer ./saved_model dizinine kaydedildi.
